In [1]:
import pandas as pd
import numpy as np

In [2]:
walmart = pd.read_csv("Walmart_Store_sales.csv")

In [3]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


Which store has maximum sales

In [42]:
walmart.groupby("Store").sum().Weekly_Sales.sort_values(ascending = False).apply(lambda x: '%.3f' %x).index[0]

20

Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation

In [43]:
walmart.groupby("Store").std().Weekly_Sales.sort_values(ascending = False).index[0]

14

In [44]:
sd = walmart.groupby("Store").std().Weekly_Sales.sort_values(ascending = False)
mean = walmart.groupby("Store").mean().Weekly_Sales.sort_values(ascending = False)
cv = sd/mean
cv

Store
1     0.100292
2     0.123424
3     0.115021
4     0.127083
5     0.118668
6     0.135823
7     0.197305
8     0.116953
9     0.126895
10    0.159133
11    0.122262
12    0.137925
13    0.132514
14    0.157137
15    0.193384
16    0.165181
17    0.125521
18    0.162845
19    0.132680
20    0.130903
21    0.170292
22    0.156783
23    0.179721
24    0.123637
25    0.159860
26    0.110111
27    0.135155
28    0.137330
29    0.183742
30    0.052008
31    0.090161
32    0.118310
33    0.092868
34    0.108225
35    0.229681
36    0.162579
37    0.042084
38    0.110875
39    0.149908
40    0.123430
41    0.148177
42    0.090335
43    0.064104
44    0.081793
45    0.165613
Name: Weekly_Sales, dtype: float64

In [45]:
from scipy.stats import variation

In [46]:
for i in walmart.Store.unique():
    print(i, end = ": ")
    print(variation(walmart[walmart.Store==i].Weekly_Sales))

1: 0.0999408357081771
2: 0.12299156720564293
3: 0.1146185290319992
4: 0.1266374152725469
5: 0.11825278836063026
6: 0.1353471207557374
7: 0.1966135996968848
8: 0.11654318873634893
9: 0.12645099879377864
10: 0.15857610369755093
11: 0.12183359468888892
12: 0.13744221956273658
13: 0.13204948069400355
14: 0.1565863429488463
15: 0.192706633729916
16: 0.16460208695319026
17: 0.12508101797248644
18: 0.16227510863890038
19: 0.13221538496995303
20: 0.13044418093151547
21: 0.16969591961518754
22: 0.15623372216503525
23: 0.17909165111910935
24: 0.12320431967333707
25: 0.1593004693299032
26: 0.10972498522926637
27: 0.13468204440572068
28: 0.1368487253164096
29: 0.18309888416863046
30: 0.05182587323723126
31: 0.08984525112455868
32: 0.1178960929691217
33: 0.0925430687099248
34: 0.10784616451867972
35: 0.22887662416388083
36: 0.16200945742452347
37: 0.041936713528847466
38: 0.11048709039759921
39: 0.1493827183948949
40: 0.12299745115463655
41: 0.1476581016905653
42: 0.09001891616458202
43: 0.06387909

---

Which store/s has good quarterly growth rate in Q3’2012?

In [47]:
walmart.Date = pd.to_datetime(walmart.Date, format = "%d-%m-%Y")

In [48]:
x = walmart.Date[0]

In [49]:
walmart["Month"] = walmart.Date.apply(lambda x: x.month)

In [50]:
walmart["Year"] = walmart.Date.apply(lambda x: x.year)

In [51]:
import datetime

In [52]:
walmart["Quarters"]= pd.cut(walmart.Month, bins = [1,3,6,9,12], labels=["Q1","Q2","Q3","Q4"], include_lowest=True)

In [53]:
quarterly_sales = walmart[walmart['Year']==2012].groupby(["Store", "Quarters"]).sum().Weekly_Sales.unstack()
q = quarterly_sales[["Q2", "Q3"]]
q[:3]

Quarters,Q2,Q3
Store,,
1,20978760.12,20253947.78
2,25083604.88,24303354.86
3,5620316.49,5298005.47


In [54]:
q.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 1 to 45
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Q2      45 non-null     float64
 1   Q3      45 non-null     float64
dtypes: float64(2)
memory usage: 1.1 KB


In [55]:
q.columns = q.columns.add_categories(['Growth'])

In [56]:
q.columns

CategoricalIndex(['Q2', 'Q3'], categories=['Q1', 'Q2', 'Q3', 'Q4', 'Growth'], ordered=True, name='Quarters', dtype='category')

In [57]:
q['Growth'] = (q['Q3'] - q['Q2'])*100/q['Q2']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
q['Growth'].sort_values(ascending = False)

Store
7     13.330776
16     8.488378
35     4.466637
26     3.955478
39     2.478404
41     2.456980
44     2.434638
24     1.652088
40     1.142841
23     0.825395
38    -0.575326
32    -0.598751
19    -0.891505
17    -1.055779
37    -1.413736
8     -1.431909
11    -1.525186
27    -1.918313
43    -2.059831
13    -2.176842
25    -2.294647
20    -2.298597
4     -2.310968
31    -2.521038
30    -2.570609
28    -2.581880
34    -2.860067
21    -2.872605
18    -2.924753
10    -3.002523
2     -3.110598
6     -3.199580
33    -3.251057
1     -3.454982
42    -3.587095
15    -4.313659
22    -4.765417
3     -5.734749
12    -6.182010
9     -6.182893
29    -6.372684
5     -6.774475
36    -7.714369
45    -7.790564
14   -15.773764
Name: Growth, dtype: float64

Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together

In [59]:
ave_Sales_NonHoliday = walmart[walmart['Holiday_Flag']==1].mean().Weekly_Sales

In [60]:
walmart.loc[(walmart['Holiday_Flag']==0) & (walmart['Weekly_Sales']>ave_Sales_NonHoliday), "Date"]

0      2010-02-05
2      2010-02-19
3      2010-02-26
4      2010-03-05
5      2010-03-12
          ...    
5861   2012-10-19
5862   2012-10-26
6337   2010-12-17
6338   2010-12-24
6390   2011-12-23
Name: Date, Length: 2418, dtype: datetime64[ns]

Provide a monthly and semester view of sales in units and give insights.

In [61]:
walmart.groupby(['Year', 'Month']).sum().Weekly_Sales

Year  Month
2010  2        1.903330e+08
      3        1.819198e+08
      4        2.314124e+08
      5        1.867109e+08
      6        1.922462e+08
      7        2.325801e+08
      8        1.876401e+08
      9        1.772679e+08
      10       2.171618e+08
      11       2.028534e+08
      12       2.887605e+08
2011  1        1.637040e+08
      2        1.863313e+08
      3        1.793564e+08
      4        2.265265e+08
      5        1.816482e+08
      6        1.897734e+08
      7        2.299114e+08
      8        1.885993e+08
      9        2.208477e+08
      10       1.832613e+08
      11       2.101624e+08
      12       2.880781e+08
2012  1        1.688945e+08
      2        1.920636e+08
      3        2.315097e+08
      4        1.889209e+08
      5        1.887665e+08
      6        2.406103e+08
      7        1.875095e+08
      8        2.368508e+08
      9        1.806455e+08
      10       1.843617e+08
Name: Weekly_Sales, dtype: float64

In [62]:
walmart.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Quarters
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2,2010,Q1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2,2010,Q1
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2,2010,Q1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2,2010,Q1
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,3,2010,Q1


In [63]:
walmart["Semesters"] = pd.cut(x = walmart.Month, bins = [1,6,12], labels = [1,2], include_lowest=True)

In [64]:
walmart[:3]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Quarters,Semesters
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2,2010,Q1,1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2,2010,Q1,1
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2,2010,Q1,1


In [65]:
walmart["Day"] = walmart.Date.apply(lambda x: x.day)

In [66]:
walmart[:1]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Quarters,Semesters,Day
0,1,2010-02-05,1643690.9,0,42.31,2.572,211.096358,8.106,2,2010,Q1,1,5


In [67]:
walmart["WeekNum"] = pd.cut(walmart.Day, bins = [1,7,14,21,28, 31], labels = [1,2,3,4,5], include_lowest=True)

In [68]:
walmart["Quarters"] = walmart.Quarters.str[1]

In [69]:
walmart[:2]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Quarters,Semesters,Day,WeekNum
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2,2010,1,1,5,1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2,2010,1,1,12,2


In [70]:
walmart.drop("Date", axis = 1, inplace = True)

In [72]:
walmart = walmart.astype(np.float64)

In [74]:
walmart[walmart['Store']==1]
#create a linear regression model on these rows

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Quarters,Semesters,Day,WeekNum
0,1.0,1643690.90,0.0,42.31,2.572,211.096358,8.106,2.0,2010.0,1.0,1.0,5.0,1.0
1,1.0,1641957.44,1.0,38.51,2.548,211.242170,8.106,2.0,2010.0,1.0,1.0,12.0,2.0
2,1.0,1611968.17,0.0,39.93,2.514,211.289143,8.106,2.0,2010.0,1.0,1.0,19.0,3.0
3,1.0,1409727.59,0.0,46.63,2.561,211.319643,8.106,2.0,2010.0,1.0,1.0,26.0,4.0
4,1.0,1554806.68,0.0,46.50,2.625,211.350143,8.106,3.0,2010.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,1.0,1437059.26,0.0,76.08,3.666,222.981658,6.908,9.0,2012.0,3.0,2.0,28.0,4.0
139,1.0,1670785.97,0.0,68.55,3.617,223.181477,6.573,10.0,2012.0,4.0,2.0,5.0,1.0
140,1.0,1573072.81,0.0,62.99,3.601,223.381296,6.573,10.0,2012.0,4.0,2.0,12.0,2.0
141,1.0,1508068.77,0.0,67.97,3.594,223.425723,6.573,10.0,2012.0,4.0,2.0,19.0,3.0
